In [1]:
import numpy as np
import tensorflow as tf

In [3]:
%run 10-decoder_layer.ipynb

4366288
(10265, 79)
(10265, 79)
(64, 79, 1024)
(64, 79, 1024)


In [13]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    x  = inputs

    #context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [14]:
num_layers = 3
d_model = 256
dff = 256
num_heads = 2
dropout_rate = 0.1

In [15]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    target_vocab_size=vocab_s,
    dropout_rate=dropout_rate)

In [17]:
output = transformer(tf.convert_to_tensor(input))

print(tf.convert_to_tensor(input).shape)
#print(pt.shape)
print(output.shape)

(64, 19)
(64, 19, 11913)
